# Exercises
file name: anomaly_detection.py or anomaly_detection.ipynb

#### Discrete data + probability
Use basic probability to identify anomalous request methods. You will want to make sure the text is normalized in order to reduce the noise.

#### Time series + EMA
Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

#### Clustering - DBSCAN
Use dbscan to detect anomalies in other products from the customers dataset.

Use dbscan to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

# %%%%%%%%%%%%%%%%%

# Anomaly Detection of Discrete Data using Probability
## Discrete data + probability

### Use basic probability to identify anomalous request methods. You will want to make sure the text is normalized in order to reduce the noise.

In [ ]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from numpy import linspace, loadtxt, ones, convolve
from sklearn.ensemble import IsolationForest

import numpy as np
import pandas as pd
import collections
import math
from datetime import datetime, timedelta

from sklearn import metrics
from random import randint
from matplotlib import style

import seaborn as sns

style.use('fivethirtyeight')
%matplotlib inline

pd.set_option('display.max_rows', 200_000)

from ipaddress import ip_address
import re
import json
from urllib.request import urlopen
# import ipinfo  # worked in Jupyter Notebook, but not working in Kaggle

### User defined function to evaluate the models.

In [ ]:
def evaluate(actual, predictions, output=True):
    mse = metrics.mean_squared_error(actual, predictions)
    rmse = math.sqrt(mse)

    if output:
        print('MSE:  {}'.format(mse))
        print('RMSE: {}'.format(rmse))
    else:
        return mse, rmse    

### User defined function to plot the models.

In [ ]:
def plot_and_eval(predictions, actual, metric_fmt='{:.2f}', linewidth=4):
    if type(predictions) is not list:
        predictions = [predictions]

    plt.figure(figsize=(16, 8))
    plt.plot(train,label='Train')
    plt.plot(test, label='Test')

    for yhat in predictions:
        mse, rmse = evaluate(actual, yhat, output=False)        
        label = f'{yhat.name}'
        if len(predictions) > 1:
            label = f'{label} -- MSE: {metric_fmt} RMSE: {metric_fmt}'.format(mse, rmse)
        plt.plot(yhat, label=label, linewidth=linewidth)

    if len(predictions) == 1:
        label = f'{label} -- MSE: {metric_fmt} RMSE: {metric_fmt}'.format(mse, rmse)
        plt.title(label)

    plt.legend(loc='best')
    plt.show()    

## Wrangle Data
### Acquire

sample data rows:

2018-01-26 09:55:03 / 1 8 97.105.19.61

2018-01-26 10:00:02 javascript-i/introduction/working-with-data-types-operators-and-variables 6 22 97.105.19.61

2018-01-26 10:00:49 javascript-i/introduction/variables 6 22 97.105.19.61

### Label each field from the curriculum log.

In [ ]:
colnames=['access_date', 'access_time', 'page_viewed', 'user_id', 'cohort_id', 'ip']
# df = pd.read_csv('./access.log',   # New data for Bayes cohort       
df = pd.read_csv('../input/anonymizedcurriculumaccesstxt/anonymized-curriculum-access.txt',          
                 engine='python',
                 header=None,
                 index_col=False,
                 names=colnames,
                 sep=' ',
#                  sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"',
#                  usecols=[0, 3, 4, 5, 6, 7, 8]
)
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

Drop rows that are missing cohort numbers.

In [ ]:
# df.cohort_id.fillna(0, inplace=True) ## had tried filling with 0... might come back and redo this...
df = df.dropna()
df.isna().sum()

In [ ]:
df.shape

Check types.

In [ ]:
df.info()

Change types to desired types.

In [ ]:
df['cohort_id'] = df.cohort_id.astype('int')
df['user_id'] = df.user_id.astype('int')

df['access_date'] = pd.to_datetime(df.access_date)
df['access_time'] = df.access_time.astype('str')
df['page_viewed'] = df.page_viewed.astype('str')
df['full_datetime'] = df.access_date.astype('str') \
                 + ' ' \
                 + df.access_time.astype('str')
df['full_datetime'] = pd.to_datetime(df['full_datetime'])

df['ip'] = df.ip.astype('str')

In [ ]:
df.dtypes

In [ ]:
df.head().append(df.tail())

#### Let's see what the date range is for these observations.

In [ ]:
df.access_date.min()

In [ ]:
df.access_date.max()

#### Our data ranges from January 26, 2018, to April 2, 2019.

#### Let's examine what kind of values we have in the "page_viewed" column.

In [ ]:
pd.options.display.max_colwidth = 100
df.page_viewed.unique().tolist()[:10]

#### We'll leave those alone for now.

### Import cohort file

In [ ]:
colnames=['cohort_id','cohort_name','start_date','end_date']
cohorts = pd.read_csv('../input/cohort-datacsv/cohort_data.csv', 
                      names=colnames,
                      header=0)     
cohorts

#### Note that cohorts 1 through 20 ended before the earliest access date in our observations.

Check the types in df and cohorts to see if anything needs to be changes.

In [ ]:
df.dtypes

In [ ]:
cohorts.dtypes

#### Change types in cohorts dataframe to match up with the df dataframe.

In [ ]:
cohorts['cohort_name'] = cohorts.cohort_name.astype('str')
cohorts['start_date']= pd.to_datetime(cohorts.start_date)
cohorts['end_date'] = pd.to_datetime(cohorts.end_date)

In [ ]:
df.dtypes

In [ ]:
cohorts.dtypes

### Merge the cohort information onto the df dataframe.

In [ ]:
df = df.merge(cohorts, on='cohort_id', how='left')
df.shape # (219070, 6)

### Let's try grouping the observations only by cohort_id.

In [ ]:
grouped_by_cohort = df.groupby('cohort_id')

In [ ]:
minimums = grouped_by_cohort.access_date.min()
maximums = grouped_by_cohort.access_date.max()
min_max_dates = pd.DataFrame()
min_max_dates['cohort_id'] = minimums.index
min_max_dates['earliest_access_date_for_cohort'] = minimums
min_max_dates['latest_access_date_for_cohort'] = maximums
min_max_dates['diff'] = maximums - minimums
min_max_dates

#### Note the no one in cohorts 1, 6, 8, 14, or 25 accessed the currilum during the timespan of our log, so let's drop those rows.

In [ ]:
min_max_dates.dropna(inplace=True)
min_max_dates.isnull().sum()

#### Check the types and then merge.

In [ ]:
min_max_dates.dtypes

In [ ]:
df = df.merge(min_max_dates, on="cohort_id")
df.head(3)

#### If the access happened after the class ended, days_after_end will be positive.
#### If the access happened before the class ended, days_after_end will be negative.

In [ ]:
df['days_after_end'] =  df.access_date - df.end_date

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df[['cohort_name', 'user_id', 'access_date', 'end_date', 'days_after_end']]

In [ ]:
df.cohort_name.unique()

In [ ]:
df.groupby(['cohort_name'])['days_after_end'].max().sort_values()

Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

In [ ]:
df.columns

In [ ]:
df.groupby(['cohort_name', 'end_date', 'access_date', 'user_id'])['page_viewed'].count()

In [ ]:
df.cohort_id.nunique()

In [ ]:
df[df.days_after_end > timedelta(days=134)]

In [ ]:
late_access = df[df.days_after_end > timedelta(days=134)]

In [ ]:
late_access.groupby('user_id')['access_date'].count().sort_values(ascending=False)

In [ ]:
max_late_access = late_access.groupby('user_id')['access_date'].count().sort_values(ascending=False)
late_users = max_late_access[:5]
late_users

# The users with user_ids 11, 314, 1, 68, and 64 were the users who accessed the curriculum over 1000 times each and over 134 days after their cohorts ended.

# The following is a list of their accesses.

In [ ]:
max_late_user_df = df[df['user_id'].isin([11, 314, 1, 68, 64])]

In [ ]:
max_late_user_df.sort_values(by='user_id')